# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

1. OpenAI takes a few minutes to register after you set up an account. If you receive an error about being over quota, try waiting a few minutes and try again.
2. Also, double check you have the right kind of API token with the right permissions. You should find it on [this webpage](https://platform.openai.com/api-keys) and it should show with Permissions of "All". If not, try creating another key by:
- Pressing "Create new secret key" on the top right
- Select **Owned by:** you, **Project:** Default project, **Permissions:** All
- Click Create secret key, and use that new key in the code and the `.env` file (it might take a few minutes to activate)
4. As a fallback, replace the line `openai = OpenAI()` with `openai = OpenAI(api_key="your-key-here")` - while it's not recommended to hard code tokens in Jupyter lab, because then you can't share your lab with others, it's a workaround for now
5. Contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point.

In [3]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

## init connexion to open ai
openai = OpenAI() 

In [4]:
# A class to capture the represented Webpage we're going to capture

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):

        #takes the url
            self.url = url

        #the requests method retrieves the web page at that url
            response = requests.get(url)

        # BeautifulSoup (BS) parses the content of the web page
            soup = BeautifulSoup(response.content, 'html.parser')

        # BS sets the title to be the web page title
            self.title = soup.title.string if soup.title else "No title found"

        # BS strips out some irrelevant sections of the web page, like JavaScript and styling information
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()

        # set the text property to be the text of all of the remaining elements in that web page.
            self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
# Let's try one out

ed = Website("https://en25minutes.fr")
print(ed.title)
print(ed.text)

En 25 minutes
En 25 minutes
Sample Page
Un engagement en faveur de l’innovation et de la durabilité
Études est un cabinet pionnier qui fusionne harmonieusement la créativité et la fonctionnalité pour redéfinir l’excellence architecturale.
À propos
Une passion pour la création d’espaces
Notre gamme complète de services professionnels s’adresse à une clientèle diversifiée, allant des propriétaires aux promotrices ou promoteurs commerciaux.
Rénovation et restauration
Découvrez la fusion de l’imagination et de l’expertise avec Études Architectural Solutions.
Support continu
Découvrez la fusion de l’imagination et de l’expertise avec Études Architectural Solutions.
Accès à l‘application
Découvrez la fusion de l’imagination et de l’expertise avec Études Architectural Solutions.
Conseil
Découvrez la fusion de l’imagination et de l’expertise avec Études Architectural Solutions.
Gestion de projet
Découvrez la fusion de l’imagination et de l’expertise avec Études Architectural Solutions.
Archite

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [ ]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edwarddonner.com")

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://anthropic.com")